In [1]:
import dataclasses

@dataclasses.dataclass
class Hyperparameter:
    batch_size: int = 32
    num_workers: int = 2
    
    
hp = Hyperparameter

In [4]:
import torch
import torchvision
import torchvision.transforms as T


trainsform = T.Compose([
    T.ToTensor(),
    T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = torchvision.datasets.CIFAR10(root='/Users/noel/data', train=True, download=True, transform=trainsform)
test_dataset = torchvision.datasets.CIFAR10(root='/Users/noel/data', train=False, download=True, transform=trainsform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=hp.batch_size, shuffle=True, num_workers=hp.num_workers)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=hp.batch_size, shuffle=True, num_workers=hp.num_workers)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
import torch.nn as nn
from enum import IntEnum

class ShiftType(IntEnum):
    DOWN = 0
    RIGHT = 1


class DownShift(nn.Module):
    def __init__(self):
        super(DownShift, self).__init__()
        
    def forward(self, x):
        
